### **Problema 1**

Primero completaremos una implementación de una estructura de conjuntos disjuntos (union-find) con compresión de rango.

In [ ]:
class DisjointForests:
    def __init__(self, n):
        assert n >= 1, 'Un bosque disjunto vacío no está permitido'
        self.n = n
        self.parents = [None] * n
        self.rank = [None] * n

    # Función: dictionary_of_sets
    # Convierte la estructura de bosque disjunto en un diccionario d
    # en el que d tiene una entrada para cada representante i
    # d[i] asigna a cada elemento que pertenece al árbol correspondiente a i
    # en el bosque disjunto.
    def dictionary_of_sets(self):
        d = {}
        for i in range(self.n):
            if self.is_representative(i):
                d[i] = set([i])
        for j in range(self.n):
            if self.parents[j] is not None:
                root = self.find(j)
                assert root in d
                d[root].add(j)
        return d

    def make_set(self, j):
        assert 0 <= j < self.n
        assert self.parents[j] is None, 'Llamas a make_set en un elemento varias veces, lo cual no está permitido.'
        self.parents[j] = j
        self.rank[j] = 1

    def is_representative(self, j):
        return self.parents[j] == j

    def get_rank(self, j):
        return self.rank[j]

    # Función: find
    # Implementa el algoritmo find para un nodo j en el conjunto.
    # Recorre repetidamente el puntero de padre hasta llegar a una raíz.
    # Implementa la estrategia de "compresión de caminos" haciendo que todos
    # los nodos a lo largo del camino desde j hasta la raíz apunten directamente a la raíz.
    def find(self, j):
        assert 0 <= j < self.n
        assert self.parents[j] is not None, 'Llamas a find en un elemento que aún no forma parte del conjunto. Por favor, llama primero a make_set.'
        if self.parents[j] != j:
            self.parents[j] = self.find(self.parents[j])  # Encuentra la raíz y comprime el camino
        return self.parents[j]

    # Función: union
    # Calcula la unión de j1 y j2
    # Primero realiza un find para obtener los representantes de j1 y j2.
    # Si no son iguales, entonces
    # implementa la unión utilizando la estrategia por rango, es decir, la raíz con rango menor se convierte
    # en hijo de la raíz con rango mayor.
    # En caso de empate, se elige que la raíz del primer argumento (j1) sea el padre.
    def union(self, j1, j2):
        assert 0 <= j1 < self.n
        assert 0 <= j2 < self.n
        assert self.parents[j1] is not None
        assert self.parents[j2] is not None
        root1 = self.find(j1)
        root2 = self.find(j2)
        if root1 != root2:
            if self.rank[root1] > self.rank[root2]:
                self.parents[root2] = root1
            elif self.rank[root1] < self.rank[root2]:
                self.parents[root1] = root2
            else:
                self.parents[root2] = root1
                self.rank[root1] += 1


In [ ]:
d = DisjointForests(10)
for i in range(10):
    d.make_set(i)

for i in range(10):
    assert d.find(i) == i, f'Fallo: find en {i} debe devolver {i}'
    
d.union(0, 1)
d.union(2, 3)
assert(d.find(0) == d.find(1)), '0 y 1 han sido unidos'
assert(d.find(2) == d.find(3)), '2 y 3 han sido unidos'
assert(d.find(0) != d.find(3)), '0 y 3 deberían estar en árboles diferentes'
assert((d.get_rank(0) == 2 and d.get_rank(1) == 1) or 
       (d.get_rank(1) == 2 and d.get_rank(0) == 1)), 'Uno de los nodos 0 o 1 debe tener rango 2'

assert((d.get_rank(2) == 2 and d.get_rank(3) == 1) or 
       (d.get_rank(3) == 2 and d.get_rank(2) == 1)), 'Uno de los nodos 2 o 3 debe tener rango 2'

d.union(3, 4)
assert(d.find(2) == d.find(4)), '2 y 4 deben estar en el mismo conjunto'

d.union(5, 7)
d.union(6, 8)
d.union(3, 7)
d.union(0, 6)

assert(d.find(6) == d.find(1)), '1 y 6 deben estar en el mismo conjunto'
assert(d.find(7) == d.find(4)), '7 y 4 deben estar en el mismo conjunto'
print('Todas las pruebas han pasado!')


### **Problema 2**

Ahora exploraremos cómo encontrar los componentes fuertemente conexos máximos de un grafo no dirigido utilizando estructuras de datos union‐find.
El grafo no dirigido consiste simplemente en una lista de aristas con pesos.
  - Asociaremos un peso no negativo $w_{i,j}$ para cada arista no dirigida $(i,j)$.
  - Asociamos algunos datos adicionales con los vértices que serán útiles más adelante.

Por favor, revisa cuidadosamente el código de las estructuras de datos para grafos no dirigidos.

In [ ]:
class UndirectedGraph:
    
    # n es el número de vértices
    # Etiquetaremos los vértices de 0 a self.n - 1
    # Simplemente almacenamos las aristas en una lista.
    def __init__(self, n):
        assert n >= 1, 'Estás creando un grafo vacío, lo cual no está permitido'
        self.n = n
        self.edges = []
        self.vertex_data = [None] * self.n
       
    def set_vertex_data(self, j, dat):
        assert 0 <= j < self.n
        self.vertex_data[j] = dat
        
    def get_vertex_data(self, j):
        assert 0 <= j < self.n
        return self.vertex_data[j]
        
    def add_edge(self, i, j, wij):
        assert 0 <= i < self.n
        assert 0 <= j < self.n
        assert i != j
        # Asegúrate de agregar la arista de i a j con peso wij
        self.edges.append((i, j, wij))
        
    def sort_edges(self):
        # Ordena las aristas en orden ascendente según sus pesos.
        self.edges = sorted(self.edges, key=lambda edg_data: edg_data[2])


#### **2A: Utiliza estructuras de datos union‐find para calcular los componentes fuertemente conexos**

Hemos visto previamente cómo usar DFS para encontrar los componentes fuertemente conexos máximos con una pequeña modificación.

  - Consideraremos únicamente aquellas aristas $(i,j)$ cuyos pesos sean menores o iguales a un umbral $W$ proporcionado por el usuario.
  - Las aristas con peso mayor a este umbral no se consideran.
  
Diseña un algoritmo para calcular todos los componentes fuertemente conexos máximos para todas las aristas con el umbral $W$ usando la estructura union‐find. ¿Cuál es el tiempo de ejecución de tu algoritmo? Nota: esta respuesta se evalúa manualmente; puedes comparar tu solución con nuestra solución proporcionada al final de esta asignación.

Tu respuesta aqui

Completa las partes faltantes de la función en el código a continuación para calcular los componentes fuertemente conexos.

In [ ]:
def compute_scc(g, W):
    # Crea un bosque disjunto con tantos elementos como vértices
    # A continuación, calcula los componentes fuertemente conexos utilizando la estructura de datos de bosque disjuntos
    d = DisjointForests(g.n)
    # Tu código aquí
    
    # Extrae un conjunto de conjuntos de d
    return d.dictionary_of_sets()    

In [ ]:
g3 = UndirectedGraph(8)
g3.add_edge(0, 1, 0.5)
g3.add_edge(0, 2, 1.0)
g3.add_edge(0, 4, 0.5)
g3.add_edge(2, 3, 1.5)
g3.add_edge(2, 4, 2.0)
g3.add_edge(3, 4, 1.5)
g3.add_edge(5, 6, 2.0)
g3.add_edge(5, 7, 2.0)
res = compute_scc(g3, 2.0)
print('Los componentes fuertemente conexos con umbral 2.0 calculados por tu código son:')
assert len(res) == 2, f'Se esperaban 2 componentes fuertemente conexos pero se obtuvo {len(res)}'
for (k, s) in res.items():
    print(s)
    
# Comprobemos que tu código devuelve lo que esperamos.
for (k, s) in res.items():
    if k in [0, 1, 2, 3, 4]:
        assert s == set([0, 1, 2, 3, 4]), '{0,1,2,3,4} debería ser un componente fuertemente conexo'
    if k in [5, 6, 7]:
        assert s == set([5, 6, 7]), '{5,6,7} debería ser un componente fuertemente conexo'

# Comprobemos que el umbral funciona correctamente
print('Componentes fuertemente conexos con umbral 1.5')
res2 = compute_scc(g3, 1.5)  # Esto descarta las aristas (2,4) y (3,4)
for (k, s) in res2.items():
    print(s)
assert len(res2) == 4, f'Se esperaban 4 componentes fuertemente conexos pero se obtuvo {len(res2)}'

for (k, s) in res2.items():
    if k in [0, 1, 2, 3, 4]:
        assert s == set([0, 1, 2, 3, 4]), '{0,1,2,3,4} debería ser un componente fuertemente conexo'
    if k == 5:
        assert s == set([5]), '{5} debería ser un componente fuertemente conexo con un solo nodo.'
    if k == 6:
        assert s == set([6]), '{6} debería ser un componente fuertemente conexo con un solo nodo.'
    if k == 7:
        assert s == set([7]), '{7} debería ser un componente fuertemente conexo con un solo nodo.'
        
print('Todas las pruebas han pasado!')


#### **2B: Calcular el árbol de expansión mínima**

Ahora calcularemos el MST (árbol de expansión mínima) de un grafo no dirigido ponderado utilizando el algoritmo de Kruskal.
Completa el siguiente código que utiliza una estructura de datos de bosque de conjuntos disjuntos para implementar el algoritmo de Kruskal.

Tu código simplemente debe devolver una lista de aristas (i, j, wij) que forman parte del MST, junto con el peso total del MST.

In [ ]:
def compute_mst(g):
    # Devuelve una tupla de dos elementos:
    #   1. lista de aristas (i,j) que forman parte del MST
    #   2. suma de los pesos de las aristas del MST.
    d = DisjointForests(g.n)
    mst_edges = []
    g.sort_edges()
    # Tu código aquí
        
    # Itera sobre las aristas ordenadas
    for edge in g.edges:
        u, v, weight = edge
        # Verifica si u y v pertenecen a conjuntos diferentes (para evitar ciclos)
        if d.find(u) != d.find(v):
            # Si están en conjuntos diferentes, incluye esta arista en el MST
            mst_edges.append((u, v, weight))
            total_weight += weight  
            
            # Une los conjuntos de u y v
            d.union(u, v)
    
    return (mst_edges, total_weight)


In [ ]:
g3 = UndirectedGraph(8)
g3.add_edge(0, 1, 0.5)
g3.add_edge(0, 2, 1.0)
g3.add_edge(0, 4, 0.5)
g3.add_edge(2, 3, 1.5)
g3.add_edge(2, 4, 2.0)
g3.add_edge(3, 4, 1.5)
g3.add_edge(5, 6, 2.0)
g3.add_edge(5, 7, 2.0)
g3.add_edge(3, 5, 2.0)

(mst_edges, mst_weight) = compute_mst(g3)
print('Tu código calculó el MST: ')
for (i, j, wij) in mst_edges:
    print(f'\t {(i, j)} peso {wij}')
print(f'Peso total de las aristas: {mst_weight}')

assert mst_weight == 9.5, 'Se esperaba que el peso óptimo del MST fuera 9.5'

assert (0, 1, 0.5) in mst_edges
assert (0, 2, 1.0) in mst_edges
assert (0, 4, 0.5) in mst_edges
assert (5, 6, 2.0) in mst_edges
assert (5, 7, 2.0) in mst_edges
assert (3, 5, 2.0) in mst_edges
assert (2, 3, 1.5) in mst_edges or (3, 4, 1.5) in mst_edges

print('Todas las pruebas han pasado!')


#### **2C: Umbral de aristas para desconectar un grafo**

Sea $G$ un grafo no dirigido ponderado que es fuertemente conexo (es decir, el grafo completo es un componente fuertemente conexo). Nuestro objetivo es encontrar el mayor peso $W$ tal que eliminar todas las aristas con peso $\geq W$ desconecte el grafo.

Demuestra que el umbral $W$ es igual al mayor peso de una arista en el MST encontrado por el algoritmo de Kruskal, demostrando que:
  - Eliminar todas las aristas con peso $\geq W$ resultará en un grafo desconectado.
  - Conservar únicamente las aristas con peso $\leq W$ (o eliminar las aristas con peso $> W$) dará como resultado un grafo conexo.

Utiliza el hecho de que un grafo es fuertemente conexo si y solo si tiene un árbol de expansión mínima.

Tu respuesta aquí

#### Análisis topológico de datos en imágenes

Ilustramos una conexión interesante entre los algoritmos de grafos para componentes fuertemente conexos y árboles de expansión mínima para analizar imágenes. Específicamente, identificaremos los componentes en las imágenes de la siguiente manera:

a) Primero, tratamos una imagen almacenada en un archivo `.png` o `.jpg` como una matriz de píxeles donde los píxeles tienen color e intensidad.

b) Dada una imagen, construimos un grafo cuyos vértices son píxeles y las aristas conectan píxeles vecinos.

c) El peso de una arista en el grafo que conecta píxeles vecinos mide la diferencia de intensidad entre los píxeles (también se pueden usar otras medidas de las diferencias locales).

Podemos realizar el siguiente análisis (esto es solo un ejemplo de este tipo de análisis, que pertenece a una familia más amplia de métodos llamados análisis topológico de datos):

(a) Construye un árbol de expansión mínima y calcula el peso máximo de una arista en el MST. Llamémoslo W.

(b) Considera los componentes fuertemente conexos máximos de la imagen para varios umbrales, tales como $0.5W$, $0.75W$ o $0.9W$. Visualizar los píxeles en los distintos componentes fuertemente conexos nos permitirá estudiar los "segmentos" que componen la imagen.

Aquí hay un poco de código útil usando OpenCV para cargar imágenes. Por favor, obsérvalo detenidamente.

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import cv2
# Puedes leer archivos .png, .jpg y otros tipos de archivo
img = cv2.imread('test-pic.png')  # Lee una imagen de un archivo usando la librería OpenCV (cv2)
# Puedes anotar imágenes
plt.imshow(img)  # Muestra la imagen en pantalla
# Puedes averiguar el tamaño de la imagen
print('Tamaño de la imagen (alto, ancho, número de capas) es', img.shape)

px = img[145,67]  # img[y,x] es el color del píxel en (x,y)
print(f'El píxel en (145,67) es {px}')
print('Los píxeles son valores RGB.')


In [ ]:
import math
import cv2

def pixel_difference(px1, px2):
    def fix_pixels(px):
        return [int(px[0]), int(px[1]), int(px[2])]
    px1_float = fix_pixels(px1)
    px2_float = fix_pixels(px2)
    return max(abs(px1_float[0] - px2_float[0]), abs(px1_float[1] - px2_float[1]), abs(px1_float[2] - px2_float[2]))

def get_index_from_pixel(i, j, height, width):
    assert 0 <= i < width
    assert 0 <= j < height
    return j * width + i

def get_coordinates_from_index(s, height, width):
    assert 0 <= s < height * width
    j = s // width
    i = s % width
    return (i, j)

def connect_neighboring_pixels(i, j, i1, j1, img, g):
    (height, width, _) = img.shape
    s = get_index_from_pixel(i, j, height, width)
    px = img[j, i]
    s1 = get_index_from_pixel(i1, j1, height, width)
    px1 = img[j1, i1]
    w = pixel_difference(px1, px)
    g.add_edge(s, s1, w)

def load_image_and_make_graph(imfilename):
    img = cv2.imread(imfilename)
    (height, width, num_layers) = img.shape
    g = UndirectedGraph(height * width)
    for j in range(height):
        for i in range(width):
            s = get_index_from_pixel(i, j, height, width)
            g.set_vertex_data(s, (i, j))
            if i > 0:
                connect_neighboring_pixels(i, j, i-1, j, img, g)
            if i < width - 1:
                connect_neighboring_pixels(i, j, i+1, j, img, g)
            if j > 0:
                connect_neighboring_pixels(i, j, i, j-1, img, g)
            if j < height - 1:
                connect_neighboring_pixels(i, j, i, j+1, img, g)
    return g


In [ ]:
print('Cargando imagen y construyendo grafo.')
g = load_image_and_make_graph('test-pic.png')
print('Ejecutando algoritmo del MST')
(mst_edges, mst_weight) = compute_mst(g)
print(f'Se encontró un MST con {len(mst_edges)} aristas y peso total = {mst_weight}')
max_mst_edge_weight = max(mst_edges, key=lambda e: e[2])
print(f'Mayor peso de arista en el MST = {max_mst_edge_weight[2]}')


In [ ]:
import numpy as np

def visualize_components(orig_image, g, components_dict):
    # Dada una imagen original, el grafo g y un diccionario de componentes,
    # crea una nueva imagen coloreada con el color de cada componente.
    (w, h, channels) = orig_image.shape
    new_image = np.zeros((w, h, channels), np.uint8)
    count = 0
    delta = 10
    for (key, vertSet) in components_dict.items():
        if len(vertSet) >= 10:
            (i, j) = g.get_vertex_data(key)
            rgb_px = orig_image[j, i]
            rgb_color = (int(rgb_px[0]), int(rgb_px[1]), int(rgb_px[2]))
            count = count + 1          
            for s in vertSet:
                (i, j) = g.get_vertex_data(s)
                cv2.circle(new_image, (i, j), 1, rgb_color, -1) 
    return new_image


In [ ]:
W0 = 0.01 * max_mst_edge_weight[2]
res = compute_scc(g, W0)
print(f'Se encontraron {len(res)} componentes')
print('Mostrando componentes con al menos 10 vértices')
new_img = visualize_components(img, g, res)
plt.imshow(new_img)  # Muestra la imagen en pantalla